<a href="https://colab.research.google.com/github/Toyonaga15/Python-for-finance/blob/main/Filter_for_analyzing_the_opening_of_a_specific_asset_after_large_drops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Filter for analyzing the opening of a specific asset when it falls less than -10%, the study was performed with the asset TESLA.

In [2]:
!pip install pandas numpy
!pip install datetime
!pip install matplotlib
!pip install yfinance

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 KB 9.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Define start and end dates for data retrieval
start_date = '2015-01-01'
end_date = '2023-01-01'

# Download historical data for the stock 'TSLA' from yfinance
tesla = yf.download('TSLA', start=start_date, end=end_date, interval='1d')

# Print the last 5 rows of data to check for correctness
print(tesla.tail())



[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-12-23  126.370003  128.619995  121.019997  123.150002  123.150002   
2022-12-27  117.500000  119.669998  108.760002  109.099998  109.099998   
2022-12-28  110.349998  116.269997  108.239998  112.709999  112.709999   
2022-12-29  120.389999  123.570000  117.500000  121.820000  121.820000   
2022-12-30  119.949997  124.480003  119.750000  123.180000  123.180000   

               Volume  
Date                   
2022-12-23  166989700  
2022-12-27  208643400  
2022-12-28  221070500  
2022-12-29  221923300  
2022-12-30  157304500  


In [4]:
# Calculate the percentage change in closing price from the previous day
tesla['Closing Percentage']= tesla['Close'].pct_change()*100

# Calculate the percentage change in opening price from the next day
tesla['Next Day Opening Percentage'] = ((tesla['Open'].shift(-1)-tesla['Close'])/tesla['Close'])*100

# Define a filter threshold for closing percentage
f = -10
filter = tesla.loc[tesla['Closing Percentage']<=(f)]

# Drop unnecessary columns
filter.drop(['High'], axis=1, inplace=True)
filter.drop(['Low'], axis=1, inplace=True)
filter.drop(['Volume'], axis=1, inplace=True)
filter.drop(['Adj Close'], axis=1, inplace=True)

#Create a list to store the result of the next day opening percentage
result = filter['Next Day Opening Percentage']
result_binary = []

# Iterate over the results and label them as positive or negative
for i in result:
  if i > 0:
    result_binary.append('Positive')
  else:
    result_binary.append('Negative')

# Add the binary labels as a new column
filter['Result'] = result_binary

#Count the number of positive results
c = 0
for i in filter['Result']:
  if i =='Positive':
    c=c+1
    
# Calculate the probability of the stock opening positively
stat = (c/len(filter['Result']))*100

# Print the final result
print(f'According to the statistics, the stock opened positively {round(stat,2)}% of the time when it closed below -10% in the selected period..\n')

According to the statistics, the stock opened positively 52.94% of the time when it closed below -10% in the selected period..



/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-4-46db4a728ac0>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter['Result'] = result_binary
